In [9]:
import numpy as np
from ppopt.mplp_program import MPLP_Program
from ppopt.mpmodel import MPModeler
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
from ppopt.plot import parametric_plot
from numpy.polynomial.legendre import leggauss
from scipy.optimize import linprog
from typing import Union
from collections import defaultdict

In [10]:
# Pistikopoulos and Mazzuchi (1990) Example 1
t_bounds = [(610, 630), (378, 398), (573, 593), (303, 323)]
# t_bounds = [(586.68, 653.32), (354.68, 421.32), (549.68, 616.32), (279.68, 346.32)]
nt = len(t_bounds)
nd = 0
d_bounds = None
d_vector = None
m = MPModeler()

t3_max = m.add_var('t3_max')
t3_min = m.add_var('t3_min')
t4_a = m.add_var('t4_a')
t4_b = m.add_var('t4_b')

t1 = m.add_param(name="t1")
t2 = m.add_param(name="t2")

m.add_constr(-1/3*t3_max + t4_a - (125 + 1/3) <= 0)
m.add_constr(-3/7*t1 - 1/7*t2 - 4/7*t3_max + (643 + 3/7) <= 0)
m.add_constr(1/2*t4_a - (161 + 1/2) <= 0)
m.add_constr(-3/5*t1 - 1/5*t2 - 2/5*t3_max - 4/5*t4_a + 922 <= 0)

m.add_constr(-1/3*t3_min + t4_b - (125 + 1/3) <= 0)
m.add_constr(-3/7*t1 - 1/7*t2 - 4/7*t3_min + (643 + 3/7) <= 0)
m.add_constr(1/2*t4_b - (161 + 1/2) <= 0)
m.add_constr(-3/5*t1 - 1/5*t2 - 2/5*t3_min - 4/5*t4_b + 922 <= 0)

m.add_constr(t3_min <= t3_max)

m.add_constr(t_bounds[2][0] <= t3_max)
m.add_constr(t_bounds[2][0] <= t3_min)
m.add_constr(t_bounds[3][0] <= t4_a)
m.add_constr(t_bounds[3][0] <= t4_b)
m.add_constr(t3_max <= t_bounds[2][1])
m.add_constr(t3_min <= t_bounds[2][1])
m.add_constr(t4_a <= t_bounds[3][1])
m.add_constr(t4_b <= t_bounds[3][1])

m.add_constr(t_bounds[0][0] <= t1)
m.add_constr(t_bounds[1][0] <= t2)
m.add_constr(t1 <= t_bounds[0][1])
m.add_constr(t2 <= t_bounds[1][1])
m.set_objective(t3_min-t3_max)
prob = m.formulate_problem()
prob.process_constraints()

solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)

def joint_pdf(theta:list):
    return (1/(5000*np.pi))*np.exp(-0.08*((theta[0]-620)**2 + (theta[1]-388)**2))

Using a found active set [10, 11, 12, 13]


In [11]:
prob.A

array([[-0.33333333,  0.        ,  1.        ,  0.        ],
       [-0.57142857,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.5       ,  0.        ],
       [-0.4       ,  0.        , -0.8       ,  0.        ],
       [ 0.        , -0.33333333,  0.        ,  1.        ],
       [ 0.        , -0.57142857,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        , -0.4       ,  0.        , -0.8       ],
       [-1.        ,  1.        ,  0.        ,  0.        ],
       [-1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.

In [12]:
prob.b

array([[ 125.33333333],
       [-643.42857143],
       [ 161.5       ],
       [-922.        ],
       [ 125.33333333],
       [-643.42857143],
       [ 161.5       ],
       [-922.        ],
       [  -0.        ],
       [-573.        ],
       [-573.        ],
       [-303.        ],
       [-303.        ],
       [ 593.        ],
       [ 593.        ],
       [ 323.        ],
       [ 323.        ]])

In [13]:
prob.F

array([[0.        , 0.        ],
       [0.42857143, 0.14285714],
       [0.        , 0.        ],
       [0.6       , 0.2       ],
       [0.        , 0.        ],
       [0.42857143, 0.14285714],
       [0.        , 0.        ],
       [0.6       , 0.2       ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [14]:
prob.A_t

array([[-1.,  0.],
       [ 0., -1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [15]:
prob.b_t

array([[-610.],
       [-378.],
       [ 630.],
       [ 398.]])

In [16]:
solution_flexibility

Solution(program=<ppopt.mplp_program.MPLP_Program object at 0x10dc84520>, critical_regions=[Critical region with active set [10, 11, 12, 13]
The Omega Constraint indices are [1, 2, 3]
The Lagrange multipliers Constraint indices are []
The Regular Constraint indices are [[7], [7]]
  x(θ) = Aθ + b 
 λ(θ) = Cθ + d 
  Eθ <= f
 A = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 
 b = [[593.]
 [573.]
 [303.]
 [303.]] 
 C = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 
 d = [[1.]
 [0.]
 [0.]
 [1.]] 
 E = [[-0.9486833  -0.31622777]
 [ 0.         -1.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]] 
 f = [[-712.14492907]
 [-378.        ]
 [ 630.        ]
 [ 398.        ]], Critical region with active set [4, 10, 11, 13]
The Omega Constraint indices are [0, 1, 2, 3]
The Lagrange multipliers Constraint indices are []
The Regular Constraint indices are [[3, 4], [3, 5]]
  x(θ) = Aθ + b 
 λ(θ) = Cθ + d 
  Eθ <= f
 A = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 
 b = [[593.        ]
 [573.        ]
 [303.    